# Smali & Manifest File Analysis

# This notebook will try to analyze a directory and its SMALI and MANIFEST files. The analysis will find and print sensitive methods and packages used by the program.

# This particular notebook only does 1 directory at a time.

In [1]:
from os import walk
import pandas as pd
import os

In [2]:
def decompileAll():

    # 1: Traverse all APKs in the folder:
    dir = 'C:\\Users\\Vin\\Desktop\\python-ransomware-analyzer\\ransomware'

    for filename in os.listdir(dir):
        if filename.endswith('.apk'):
            print("decompiling: "+ filename)
            
            #set output directory (removing .apk from the end)
            foldername = 'out_' + filename[:-4]
            var1 = r'C:\Users\Vin\Desktop\python-ransomware-analyzer\decompiledApks\\' + foldername
        
            #set input apk
            var2 = r'C:\Users\Vin\Desktop\python-ransomware-analyzer\ransomware\\' + filename

            #convert python vars to batch vars
            os.putenv("VAR1", var1)
            os.putenv("VAR2", var2)

            #call the run.bat for the apktool
            os.system("run.bat")
    print("...Done!")

In [3]:
#create empty lists
methods_to_find = []
perms_to_find = []

#read file using pandas
df = pd.read_csv('functions.csv', engine='python')
methods_to_find = df.functions

df = pd.read_csv('perms.csv', engine='python')
perms_to_find   = df.perms

In [4]:
#Create class object to be used in the lists
class obj:
    def __init__(self, name):
        self.name = name
        self.count = 0
    def print(self):
        print(self.name + ": " + str(self.count))

In [5]:
#Initialize empty lists
myList = []
fakeList = []

#Fill lists with permissions and functions
for i in perms_to_find:
    myList.append(obj(i))
    fakeList.append(i)

for i in methods_to_find:
    myList.append(obj(i))
    fakeList.append(i)

#self print function for list
def printList():
    for i in range(len(myList)):
        myList[i].print()

In [6]:
#Create empty dataframe
df = pd.DataFrame(columns=fakeList)
df.to_csv('my_csv.csv', mode='w', header=True)

In [7]:
def findFunctionsInDir(direct, list=0, verbose = True):

    #for each directory, and files
    for dirpath, dirnames, files in walk(direct):

        #skip the res folder, too many subfolders, slows.
        if('res' in dirpath): continue

        if(verbose == True): print(f'Found directory: {dirpath}')
        for file_name in files:

            #if it is a smali file
            if('.smali' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                with open(dirpath + "\\" + file_name) as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in methods_to_find:
                            if(i in line and 'invoke' in line):
                                
                                #Increase counter for found item
                                myList[fakeList.index(i)].count += 1

            #if it is a manifest file
            if('.xml' in file_name and 'anifest' in file_name):
                
                #print file name
                if(verbose == True): print(file_name)

                #open file and print line number and contents
                #I had to use latin encoding here or it would simply not work.
                with open(dirpath + "\\" + file_name, encoding="latin-1") as myFile:
                    for num, line in enumerate(myFile, 1):
                        for i in perms_to_find:
                            if(i in line and ('<uses-permission android:name=' in line)
                                         or  ('<action android:name="android.app.action') in line):
                                
                                #Set permission found to 1
                                myList[fakeList.index(i)].count += 1

In [8]:
#function saves to csv file
def saveToFile(df):
    df.to_csv('my_csv.csv', mode='a', header=False)  

In [9]:
def appendToDF(myList, df):
    #initialize empty list for the counts only.
    listOfCount = []

    #fill such list
    for i in range(len(myList)):
        listOfCount.append(myList[i].count)

    #append the list to the dataframe
    df.loc[len(df.index)] = listOfCount

In [10]:
#for the passed directory, do: CALL FIND FUNCTIONS, APPEND THE RESULTING myList to the df, save.
def runAnalysis(filename):
    findFunctionsInDir(direct=filename, verbose=True)

    #see if list checks out:
    printList()

    appendToDF(myList, df)

In [11]:
#decompile all of the APKs in folder (hardcoded)
#decompileAll()

#This line will clear the DF (call if needed)
df = df.iloc[0:0]

direc = "C:\\Users\\Vin\\Desktop\\python-ransomware-analyzer\\decompiledApks"

# Here, filename means folder.
for filename in os.listdir(direc):
    print("Analyzing: " + filename)
    runAnalysis(filename)

saveToFile(df)
print("..done! Saved to csv.")

Analyzing: out_047f5f8602dbe34cb94bfc6d508e49fb
READ_CALENDAR: 0
WRITE_CALENDAR: 0
CAMERA: 0
READ_CONTACTS: 0
WRITE_CONTACTS: 0
GET_ACCOUNTS: 0
ACCESS_FINE_LOCATION: 0
ACCESS_COARSE_LOCATION: 0
RECORD_AUDIO: 0
READ_PHONE_STATE: 0
READ_PHONE_NUMBERS: 0
CALL_PHONE: 0
ANSWER_PHONE_CALLS: 0
READ_CALL_LOG: 0
WRITE_CALL_LOG: 0
ADD_VOICEMAIL: 0
USE_SIP: 0
PROCESS_OUTGOING_CALLS: 0
BODY_SENSORS: 0
SEND_SMS: 0
RECEIVE_SMS: 0
READ_SMS: 0
RECEIVE_WAP_PUSH: 0
RECEIVE_MMS: 0
READ_EXTERNAL_STORAGE: 0
WRITE_EXTERNAL_STORAGE: 0
CALL_LOG: 0
DEVICE_ADMIN_ENABLED: 0
SET_WALLPAPER: 0
USER_PRESENT: 0
QUICKBOOT_POWERON: 0
BOOT_COMPLETED: 0
REBOOT: 0
SYSTEM_ALERT_WINDOW: 0
GET_TASKS: 0
MOUNT_UNMOUNT_FILESYSTEMS: 0
sendTextMessage: 0
sendDataMessage: 0
sendMultimediaMessage: 0
sendMultipartTextMessage: 0
setReadOnly: 0
setReadable: 0
setWritable: 0
Cipher: 0
execSQL: 0
SQLiteDatabase: 0
bindSocket: 0
openConnection: 0
getFilesDir: 0
getCacheDir: 0
getExternalFilesDir: 0
getExternalCacheDir: 0
shareIntent: 0
r

In [12]:
df

,READ_CALENDAR,WRITE_CALENDAR,CAMERA,READ_CONTACTS,WRITE_CONTACTS,GET_ACCOUNTS,ACCESS_FINE_LOCATION,ACCESS_COARSE_LOCATION,RECORD_AUDIO,READ_PHONE_STATE,...,getExternalCacheDir,shareIntent,readPhoneNumber,FileInputStream,getSharedPreferences,telephony,getMessageBody,getDisplayOriginatingAddress,wallpaper,crypto
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
#This line will clear the DF (call if needed)
df = df.iloc[0:0]